In [10]:
file_path = r"C:\Users\james\Documents\GitHub\Traffic_Volume_Prediction"

In [11]:
import pandas as pd

train = pd.read_csv(file_path + r"\Data\train.csv")
test = pd.read_csv(file_path + r"\Data\test.csv")

In [12]:
# X_train과 y_train 설정
X_train = train.drop(columns=['1005004000_velocity'])
y_train = train[['1005004000_velocity']]

# X_test와 y_test 설정
X_test = test.drop(columns=['1005004000_velocity'])
y_test = test[['1005004000_velocity']]

In [13]:
# 타겟 컬럼과 시간 시차 설정
TARGET = '1005004000_velocity'
HORIZON = 24  # 24시간 시차

# 24시간 시차 적용하여 데이터 생성
train['y_shifted'] = train[TARGET].shift(-HORIZON)  # 24시간 이후 값을 타겟으로 설정
test['y_shifted'] = test[TARGET].shift(-HORIZON)

# 비어 있는 데이터 제거
train = train.dropna(subset=['y_shifted'])
test = test.dropna(subset=['y_shifted'])

# X_train, y_train 설정
X_train = train.drop(columns=[TARGET, 'y_shifted'])
y_train = train[['y_shifted']]

# X_test, y_test 설정
X_test = test.drop(columns=[TARGET, 'y_shifted'])
y_test = test[['y_shifted']]

# 결과 출력 (확인용)
print(f"X_train shape: {X_train.shape}")
print(f"y_train shape: {y_train.shape}")
print(f"X_test shape: {X_test.shape}")
print(f"y_test shape: {y_test.shape}")

X_train shape: (19261, 21)
y_train shape: (19261, 1)
X_test shape: (696, 21)
y_test shape: (696, 1)


In [14]:
# 넷 다 dataframe
print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

(19261, 21) (19261, 1) (696, 21) (696, 1)


In [15]:
# 처리 방식 정의
fill_methods = {
    '1050003300_velocity': 'ffill',
    '1070000200_velocity': 'ffill',
    '1070000200_velocity': 'ffill',
    '1070000500_velocity': 'ffill',
    '1070001600_velocity': 'ffill',
    'F-03 유입_traffic': 'ffill',
    'F-04 유출_traffic': 'ffill',
    '강수량(mm)' : 0,
    '풍속(m/s)' : 'average',
    '적설(cm)': 0
}

for df in [X_train, X_test]:
    # for loop로 결측값 처리
    for column, method in fill_methods.items():
        if method == 'average':  # 평균으로 채우기
            df[column] = df[column].fillna(df[column].mean())
        elif method == 'ffill':  # 이전 값으로 채우기
            df[column] = df[column].ffill()
        else:  # 특정 값으로 채우기
            df[column] = df[column].fillna(method)

In [16]:
# 모든 열을 float32로 변환
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

In [17]:
from sklearn.preprocessing import MinMaxScaler

# X_train과 X_test를 하나로 합침
X_combined = pd.concat([X_train, X_test], axis=0)

# MinMaxScaler 적용
scaler = MinMaxScaler()
X_combined_scaled = pd.DataFrame(scaler.fit_transform(X_combined), columns=X_combined.columns)

# 다시 분리
X_train = X_combined_scaled.iloc[:len(X_train), :].reset_index(drop=True)
X_test = X_combined_scaled.iloc[len(X_train):, :].reset_index(drop=True)

In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader
from sklearn.metrics import mean_absolute_percentage_error
import numpy as np
from sklearn.model_selection import TimeSeriesSplit

def set_seed(seed):
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

# 시드 설정
set_seed(42)

# 설정 값
N_TEST = 12
N_SPLIT = 27
TEMPORAL_FEATURES = ['1050003300_velocity', '1050020400_velocity', '1070000200_velocity',
                     '1070000500_velocity', '1070001600_velocity', 'F-03 유입_traffic',
                     'F-04 유출_traffic', '기온(°C)', '강수량(mm)', '풍속(m/s)', '습도(%)', '적설(cm)']
STATIC_FEATURES = ['Year', 'is_weekend', 'is_holiday', 'is_morning_rush', 'is_evening_rush']

INPUT_SIZE = len(TEMPORAL_FEATURES)
STATIC_SIZE = len(STATIC_FEATURES)
HIDDEN_SIZE = 128
OUTPUT_SIZE = 1
EPOCHS = 300
LEARNING_RATE = 0.001
BATCH_SIZE = 64  # 배치 사이즈 설정

# CUDA 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# TimeSeriesSplit 설정
tscv = TimeSeriesSplit(n_splits=N_SPLIT)

# MAPE 점수 리스트
mape_scores = []

# TFT 모델 정의
class TemporalFusionTransformer(nn.Module):
    def __init__(self, input_size, static_size, hidden_size, output_size):
        super(TemporalFusionTransformer, self).__init__()
        self.hidden_size = hidden_size

        # Static Covariate Encoder
        self.static_fc = nn.Linear(static_size, hidden_size)

        # Temporal Covariate Encoder
        self.temporal_fc = nn.Linear(input_size, hidden_size)

        # LSTM Layer
        self.lstm = nn.LSTM(hidden_size, hidden_size, batch_first=True)

        # Attention Layer
        self.attention = nn.MultiheadAttention(hidden_size, num_heads=4, batch_first=False)

        # Fully Connected Layer
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x_temporal, x_static):
        # Static Feature Encoding
        static_out = self.static_fc(x_static)  # [Batch, Hidden]
        # static_out을 [Batch, Sequence, Hidden]로 맞추기 위해 x_temporal의 Sequence 길이만큼 반복
        static_out = static_out.unsqueeze(1).repeat(1, x_temporal.size(1), 1)  # [Batch, Sequence, Hidden]

        # Temporal Feature Encoding
        x_temporal = self.temporal_fc(x_temporal)  # [Batch, Sequence, Hidden]

        # Combine Static and Temporal Features
        x_combined = x_temporal + static_out  # [Batch, Sequence, Hidden]

        # LSTM Layer
        lstm_out, _ = self.lstm(x_combined)  # [Batch, Sequence, Hidden]

        # Attention Layer
        # LSTM 출력 -> [Sequence, Batch, Hidden] 형태 변환
        lstm_out = lstm_out.permute(1, 0, 2)
        attention_out, _ = self.attention(lstm_out, lstm_out, lstm_out)
        # Attention 결과 -> [Batch, Sequence, Hidden] 형태로 복원
        attention_out = attention_out.permute(1, 0, 2)

        # Fully Connected Output (마지막 타임스텝 사용)
        out = self.fc(attention_out[:, -1, :]).unsqueeze(-1)  # [Batch, Output]
        return out

# 모델 초기화
model = TemporalFusionTransformer(INPUT_SIZE, STATIC_SIZE, HIDDEN_SIZE, OUTPUT_SIZE).to(device)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)

for fold, (train_idx, val_idx) in enumerate(tscv.split(X_train)):
    # 마지막 N_TEST splits 만 사용
    if fold < N_SPLIT - N_TEST:
        continue

    # Train/Validation 데이터 분리
    X_tr = X_train.iloc[train_idx][TEMPORAL_FEATURES].values
    X_val = X_train.iloc[val_idx][TEMPORAL_FEATURES].values
    X_tr_static = X_train.iloc[train_idx][STATIC_FEATURES].values
    X_val_static = X_train.iloc[val_idx][STATIC_FEATURES].values
    y_tr = y_train.iloc[train_idx].values
    y_val = y_train.iloc[val_idx].values

    # Tensor 변환 및 차원 조정 ([Batch, Sequence=1, Features])
    X_tr_tensor = torch.tensor(X_tr, dtype=torch.float32).unsqueeze(1).to(device)  # [Batch, 1, Features]
    X_val_tensor = torch.tensor(X_val, dtype=torch.float32).unsqueeze(1).to(device)
    X_tr_static_tensor = torch.tensor(X_tr_static, dtype=torch.float32).to(device)
    X_val_static_tensor = torch.tensor(X_val_static, dtype=torch.float32).to(device)
    y_tr_tensor = torch.tensor(y_tr, dtype=torch.float32).to(device).unsqueeze(1)  # [Batch, 1]
    y_val_tensor = torch.tensor(y_val, dtype=torch.float32).to(device).unsqueeze(1)

    # Dataset, DataLoader
    train_dataset = TensorDataset(X_tr_tensor, X_tr_static_tensor, y_tr_tensor)
    val_dataset = TensorDataset(X_val_tensor, X_val_static_tensor, y_val_tensor)

    train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=len(val_dataset), shuffle=False)

    # 모델 학습
    for epoch in range(EPOCHS):
        model.train()
        for X_batch, X_static_batch, y_batch in train_loader:
            optimizer.zero_grad()
            y_pred = model(X_batch, X_static_batch)
            loss = criterion(y_pred, y_batch)
            loss.backward()
            optimizer.step()

    # 검증
    model.eval()
    with torch.no_grad():
        for X_val_batch, X_val_static_batch, y_val_batch in val_loader:
            y_val_pred = model(X_val_batch, X_val_static_batch).cpu().squeeze().numpy()
            y_val_actual = y_val_batch.cpu().squeeze().numpy()
            mape = mean_absolute_percentage_error(y_val_actual, y_val_pred)
            mape_scores.append(mape)
            print(f"Fold {fold + 1}, Validation MAPE: {mape:.4f}")

# 테스트 세트에 대한 최종 예측
X_test_tensor = torch.tensor(X_test[TEMPORAL_FEATURES].values, dtype=torch.float32).unsqueeze(1).to(device)
X_test_static_tensor = torch.tensor(X_test[STATIC_FEATURES].values, dtype=torch.float32).to(device)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.float32).unsqueeze(1).to(device)

model.eval()
with torch.no_grad():
    y_test_pred = model(X_test_tensor, X_test_static_tensor).cpu().squeeze().numpy()
    y_test_actual = y_test_tensor.cpu().squeeze().numpy()
    test_mape = mean_absolute_percentage_error(y_test_actual, y_test_pred)

# 결과 출력
print("Results: ")
print(f"Average Validation MAPE: {np.mean(mape_scores):.4f}")
print(f"Validation MAPE Std Dev: {np.std(mape_scores):.4f}")
print(f"Final Test MAPE: {test_mape:.4f}")

# GPU 메모리 비우기
torch.cuda.empty_cache()
print("CUDA memory cleared.")

Using device: cuda


Using device: cuda
Fold 16, Validation MAPE: 0.1079
Fold 17, Validation MAPE: 0.1292
Fold 18, Validation MAPE: 0.1456
Fold 19, Validation MAPE: 0.1655
Fold 20, Validation MAPE: 0.1028
Fold 21, Validation MAPE: 0.0961
Fold 22, Validation MAPE: 0.0756
Fold 23, Validation MAPE: 0.0970
Fold 24, Validation MAPE: 0.0902
Fold 25, Validation MAPE: 0.0937
Fold 26, Validation MAPE: 0.1074
Fold 27, Validation MAPE: 0.0923
Results: 
Average Validation MAPE: 0.1086
Validation MAPE Std Dev: 0.0246
Final Test MAPE: 0.0878
CUDA memory cleared